In [1]:
import pandas as pd

In [5]:
df=pd.read_excel('catalog_v1.xlsx',sheet_name='1')

In [8]:
df

,№ п/п,ТНВЭД ЕАЭС,Наименование товара,Уровень передела
0,1,40210,"Молоко и сливки, сгущенные или с добавлением с...",средний
1,2,40320,Йогурт,средний
2,3,40390,"Прочие пахта, свернувшиеся молоко и сливки, ке...",средний
3,4,40410,"Молочная сыворотка, видоизмененная молочная сы...",средний
4,5,40490,Прочие продукты из натуральных компонентов мол...,средний
...,...,...,...,...
3803,3804,950662,Мячи надувные,средний
3804,3805,950691,Инвентарь и оборудование для занятий общей физ...,высокий
3805,3806,950699,Прочие снаряды и инвентарь для занятий физкуль...,высокий
3806,3807,960350,"Щетки, являющиеся частями механизмов, приборов...",высокий


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
v = TfidfVectorizer(input='content',
                    encoding='utf-8', decode_error='replace', strip_accents='unicode',
                    lowercase=True, analyzer='word', stop_words='english',
                    token_pattern=r'(?u)\b[а-яА-Я_][а-яА-Я0-9_]+\b',
                    ngram_range=(1, 2), max_features=20000,
                    norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=True,
                    max_df=0.1, min_df=5)

In [29]:
tfidf_vectorizer_vectors=v.fit_transform(df['Наименование товара'].tolist())

In [31]:
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[23]
df1=pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=v.get_feature_names(), columns=["tfidf"])
df1.sort_values(by=["tfidf"],ascending=False)

,tfidf
сырое,0.239168
фракции нерафинированные,0.234222
их химического,0.229937
без изменения,0.229937
нерафинированные,0.229937
...,...
компонентов,0.000000
компонентов для,0.000000
конвеиеры,0.000000
конденсаторы,0.000000


In [32]:
sum(first_vector_tfidfvectorizer.T.todense())

matrix([[4.7299088]])

In [36]:
df1.sort_values(by=["tfidf"],ascending=False).head(30)

,tfidf
сырое,0.239168
фракции нерафинированные,0.234222
их химического,0.229937
без изменения,0.229937
нерафинированные,0.229937
нерафинированные или,0.229937
изменения,0.229937
изменения их,0.229937
или рафинированные,0.229937
рафинированные но,0.229937


In [40]:
import re

from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

In [46]:
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

In [118]:
def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    if len(tokens) > 0:
        return tokens
    return None

In [ ]:
data = df.apply(lemmatize)

In [57]:
df[['Наименование товара']]

,Наименование товара
0,"Молоко и сливки, сгущенные или с добавлением с..."
1,Йогурт
2,"Прочие пахта, свернувшиеся молоко и сливки, ке..."
3,"Молочная сыворотка, видоизмененная молочная сы..."
4,Прочие продукты из натуральных компонентов мол...
...,...
3803,Мячи надувные
3804,Инвентарь и оборудование для занятий общей физ...
3805,Прочие снаряды и инвентарь для занятий физкуль...
3806,"Щетки, являющиеся частями механизмов, приборов..."


In [66]:
data=df['Наименование товара'].apply(lemmatize)

In [69]:
data=data.dropna()

In [90]:
data.str.join(' ').tolist()

['молоко сливка сгустить добавление сахар другой подслащивать вещество порошок гранула другой твёрдый форма содержание жир',
 'прочий пахта свернуться молоко сливка кефир прочий ферментировать сквасить молоко сливка сгустить несгустить добавление добавление сахар другой подслащивать вещество вкусый ароматический добавка добавление добавление фрукт орех какао',
 'молочный сыворотка видоизменить молочный сыворотка сгустить несгустить добавление добавление сахар другой подслащивать вещество',
 'прочий продукт натуральный компонент молоко добавление добавление сахар другой подслащивать вещество друг место поименованный включить',
 'сыр молодой недозрелый невыдержанный включая сывороточный альбуминовый сыр творог',
 'тёртый сыр сыр порошок вид',
 'плавленый сыр нетёртый порошкообразный',
 'голубой прочий сыр содержать прожилка получить использование',
 'мука тонкий грубый помол порошок картофельный',
 'хлопья гранула картофельный',
 'клейковина пшеничный сухой сырой',
 'масло соевый сырой н

In [93]:
tfidf_vectorizer_vectors=v.fit_transform(data.str.join(' ').tolist())

In [99]:
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[222]
df1=pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=v.get_feature_names(), columns=["tfidf"])
df1.sort_values(by=["tfidf"],ascending=False).head(30)

,tfidf
прочии акриловыи,0.258496
животное расфасовать,0.249442
исключительно шерсть,0.249442
модакриловыи,0.242212
акриловыи модакриловыи,0.242212
пряжа прочии,0.239078
акриловыи,0.222522
шерсть тонкии,0.217240
шерсть,0.207287
смешанныи основное,0.194812


In [122]:
pd.Series(['привет как дела 435 345 нуда еще бы приваа']).apply(lemmatize)

0    [привет, дело, нуда, приваа]
dtype: object

In [123]:
pd.Series(['привет да']).apply(lemmatize)

0    [привет]
dtype: object

['привет']

In [126]:
v.transform().T.todense()

AttributeError: 'list' object has no attribute 'lower'

In [134]:
df1=pd.DataFrame(v.transform(pd.Series(['автоматическии лом']).apply(lemmatize)[0]), index=v.get_feature_names(), columns=["tfidf"])
df1.sort_values(by=["tfidf"],ascending=False)

ValueError: Shape of passed values is (2, 1), indices imply (2012, 1)

In [139]:
pd.DataFrame(v.transform(pd.Series(['автоматическии лом']).apply(lemmatize)[0]).T.todense())

,0,1
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
2007,0.0,0.0
2008,0.0,0.0
2009,0.0,0.0
2010,0.0,0.0


In [142]:
v.transform().T.todense()

matrix([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [175]:
a=[' '.join(pd.Series(['автоматическии, лом, мыло, автомобиль, аддитивныи']).apply(lemmatize)[0])]

In [176]:
pd.DataFrame(v.transform(a).T.todense())

,0
0,0.000000
1,0.486965
2,0.000000
3,0.530870
4,0.000000
...,...
2007,0.000000
2008,0.000000
2009,0.000000
2010,0.000000


In [170]:
v.get_feature_names()

['автоматическии',
 'автомобиль',
 'агломерировать',
 'аддитивныи',
 'аддитивныи производство',
 'аккумулятор',
 'акриловыи',
 'акриловыи модакриловыи',
 'активныи',
 'активныи органическии',
 'активныи средство',
 'алкалоид',
 'алкалоид производныи',
 'альдегидныи',
 'альдегидныи смола',
 'алюминиевыи',
 'алюминиевыи сплав',
 'алюминии',
 'аммонии',
 'аналог',
 'аналогичныи',
 'аналогичныи емкость',
 'аналогичныи изделие',
 'аналогичныи инструмент',
 'аналогичныи кремнеземистыи',
 'аналогичныи материал',
 'аналогичныи нить',
 'аналогичныи продукт',
 'аналогичныи способ',
 'аналогичныи устроиство',
 'аналогичныи форма',
 'ангидрид',
 'ангидрид галогенангидрид',
 'аппарат',
 'аппарат дистанционныи',
 'аппарат кг',
 'аппаратныи',
 'аппаратныи прядение',
 'аппаратура',
 'аппаратура измерение',
 'аппаратура основать',
 'аппаратура приемныи',
 'аппаратура товарныи',
 'арматура',
 'армированныи',
 'армированныи комбинированныи',
 'ароматическии',
 'асбест',
 'ацетат',
 'ациклическии',
 'ацик

In [183]:
a=[' '.join(pd.Series(['лучевои, лом, мыло, автомобиль, аддитивныи']).apply(lemmatize)[0])]
df1=pd.DataFrame(v.transform(a).T.todense(), index=v.get_feature_names(), columns=["tfidf"])
df1.sort_values(by=["tfidf"],ascending=False).head(30)

,tfidf
аддитивныи,0.478467
мыло,0.445119
лом,0.438897
автомобиль,0.438897
лучевои,0.433217
прочии маика,0.000000
прочии коррозионностоикии,0.000000
прочии кроме,0.000000
прочии легировать,0.000000
прочии лекарственныи,0.000000
